# Basic PySyft Setup

### RecNN prerequisites 

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.tensorboard import SummaryWriter
import syft as sy
from tqdm.auto import tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


# == recnn ==
import sys
sys.path.append("../../")
import recnn

# you can enable cuda here
cuda = False
if cuda:
    cuda = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

# ---
frame_size = 10
batch_size = 25
n_epochs   = 100
plot_every = 30
step       = 0
# --- 

tqdm.pandas()

from jupyterthemes import jtplot
jtplot.style(theme='grade3')

In [3]:
def overwrite_batch_tensor_embeddings(batch, item_embeddings_tensor, frame_size):
    
    from recnn.data.utils import get_irsu
    items_t, ratings_t, sizes_t, users_t = get_irsu(batch)
    
    items_emb = item_embeddings_tensor[items_t.long()]
    b_size = ratings_t.size(0)

    items = items_emb[:, :-1, :].view(b_size, -1)
    next_items = items_emb[:, 1:, :].view(b_size, -1)
    ratings = ratings_t[:, :-1]
    next_ratings = ratings_t[:, 1:]
    
    state = torch.cat([items, ratings], 1)
    next_state = torch.cat([next_items, next_ratings], 1)
    action = items_emb[:, -1, :]
    reward = ratings_t[:, -1]

    done = torch.zeros(b_size)
    # for some reason syft dies at this line
    # so no done in training. not a big deal
    # done[torch.cumsum(sizes_t - frame_size, dim=0) - 1] = 1
    
    batch = {'state': share(state), 'action': share(action),
             'reward': share(reward), 'next_state': share(next_state),
             'done': share(done),
             'meta': {'users': users_t, 'sizes': sizes_t}}
    return batch

# overwrite batch generation function
recnn.data.utils.batch_tensor_embeddings = overwrite_batch_tensor_embeddings

In [4]:
# embeddgings: https://drive.google.com/open?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
env = recnn.data.env.FrameEnv('../../data/embeddings/ml20_pca128.pkl',
                         '../../data/ml-20m/ratings.csv', frame_size, batch_size, num_workers=0)

In [5]:
# test function
def run_tests():
    batch = next(iter(env.test_dataloader))
    loss = ddpg.update(batch, learn=False)
    return loss



value_net  = recnn.nn.Critic(1290, 128, 256, 54e-2)
policy_net = recnn.nn.Actor(1290, 128, 256, 6e-1)

if cuda:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    value_net  = recnn.nn.Critic(1290, 128, 256, 54e-2).to(cuda)
    policy_net = recnn.nn.Actor(1290, 128, 256, 6e-1).to(cuda)
    torch.set_default_tensor_type('torch.FloatTensor')

ddpg = recnn.nn.DDPG(policy_net, value_net)
ddpg.writer = SummaryWriter(log_dir='../../runs')
plotter = recnn.utils.Plotter(ddpg.loss_layout, [['value', 'policy']],)

## Syft

In [6]:
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [7]:
def share_no_cuda(m):
    m = m.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
    return m

def share_cuda(m):
    m = m.fix_precision()
    m = m.to(cuda).share(bob, alice, crypto_provider=james, requires_grad=True)
    return m

def share(m):
    if cuda:
        return share_cuda(m)
    else:
        return share_no_cuda(m)

if not cuda:
    ddpg.nets = dict([(k, share(v)) for k, v in ddpg.nets.items()])
share_optim = lambda o: o.fix_precision()
ddpg.optimizers =  dict([(k, share_optim(v)) for k, v in ddpg.optimizers.items()])

## Warning: it may freeze your system here.

On my PC it just freezes it. But google colab seem to withstand the BIG FREEZE. Just wait for 5 minutes and restart, click on stop button and restart the cell.

In [10]:
plot_every = 50
n_epochs = 2

def learn():
    for epoch in range(n_epochs):
        for batch in tqdm(env.train_dataloader):
            loss = ddpg.update(batch, learn=True)
            plotter.log_losses(loss)
            ddpg.step()
            if ddpg._step % plot_every == 0:
                clear_output(True)
                print('step', ddpg._step)
                test_loss = run_tests()
                plotter.log_losses(test_loss, test=True)
                plotter.plot_loss()
            if ddpg._step > 1000:
                return
            
learn()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dev/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-21692f94cb16>", line 19, in <module>
    learn()
  File "<ipython-input-10-21692f94cb16>", line 7, in learn
    loss = ddpg.update(batch, learn=True)
  File "../../recnn/nn/algo.py", line 67, in update
    self.device, self.debug, self.writer, step=self._step, learn=learn)
  File "../../recnn/nn/update.py", line 60, in ddpg_update
    next_action = nets['target_policy_net'](next_state)
  File "/home/dev/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "../../recnn/nn/models.py", line 65, in forward
    action = F.relu(self.linear1(state))
  File "/home/dev/.local/lib/python3.7/site-packages/syft/generic/frameworks/hook/hook.py", line 449, in overloaded_func
    response = ha

KeyboardInterrupt: 